In [ ]:
from ModelTools.Metric import Metric
from ModelTools.RegBuilder import RegBuilder
from ModelTools._src import tabulate
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
rng = np.random.default_rng(0)
x = rng.normal(0,0.1,200).cumsum()
x = np.linspace(-3,3,500)
y = rng.normal(1+2*x**2+x,scale=1)
df = pd.DataFrame({'x':x,'y':y})

m=RegBuilder(
    'fml_OLS',
    formula='~x0+I(x0**2)'
)
m.fit(x.reshape(-1,1),y)

m.coef

In [ ]:
import matplotlib.pyplot as plt

x_new = np.linspace(-3,3,500)
y_new = rng.normal(1+2*x_new**2+x_new,scale=1)

mean = m.predict(x_new.reshape(-1,1))
lw,up=m.predict_interval(x_new.reshape(-1,1),alpha=0.01)

fig,ax = plt.subplots(1,2,figsize=[10,5])

in_sample_index = (x_new<x.max()) & (x_new>x.min())
ax[0].plot(x_new[in_sample_index],mean[in_sample_index],c='red')
ax[0].scatter(x_new[in_sample_index],y_new[in_sample_index])
ax[0].fill_between(x_new[in_sample_index],lw[in_sample_index],up[in_sample_index],alpha=0.3)

ax[1].plot(x_new,mean,c='red')
ax[1].scatter(x_new,y_new,s=1)
ax[1].fill_between(x_new,lw,up,alpha=0.3)
ax[1].vlines(x=min(x),ymin=min(lw),ymax=max(up),colors='green')
ax[1].vlines(x=max(x),ymin=min(lw),ymax=max(up),colors='green')

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas
from formulaic import model_matrix
from statsmodels.api import OLS
from sklearn.linear_model import LinearRegression

# Build some data, and hard-code "y" as a quartic function with some Gaussian noise.


# Generate a model matrix with a polynomial coding in "x".
y, X = model_matrix("y ~ poly(x, degree=2,raw=True)", df, output="numpy")

# Fit coefficients for the intercept and polynomial basis
m=LinearRegression().fit(X,y)

In [ ]:
df

In [ ]:
from patsy import dmatrix,build_design_matrices

dm = dmatrix('~bs(x,3)',df)
build_design_matrices([dm.design_info],df)[0]

In [ ]:
dm.design_info.column_names